In [1]:
!pip install ortools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 24.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [7]:
import sys
import time
from ortools.sat.python import cp_model

In [8]:
model = cp_model.CpModel()

In [9]:

board_size = 4
queens = [
    model.NewIntVar(0, board_size - 1, 'x%i' % i) for i in range(board_size)
]

In [10]:
model.AddAllDifferent(queens)

model.AddAllDifferent(queens[i] + i for i in range(board_size))
model.AddAllDifferent(queens[i] - i for i in range(board_size))

In [11]:
from ortools.sat.python import cp_model

class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, queens):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.queens = queens
        self.solutions = []
        self.solution_count = 0

    def on_solution_callback(self):
        solution = [self.Value(queen) for queen in self.queens]
        self.solutions.append(solution)
        self.solution_count += 1

def solve_n_queens(board_size):
    model = cp_model.CpModel()
    queens = [model.NewIntVar(0, board_size - 1, f'queen_{i}') for i in range(board_size)]

    for i in range(board_size):
        for j in range(i + 1, board_size):
            model.Add(queens[i] != queens[j])
            model.Add(queens[i] + i != queens[j] + j)
            model.Add(queens[i] - i != queens[j] - j)

    solver = cp_model.CpSolver()
    solution_printer = SolutionPrinter(queens)
    solver.SearchForAllSolutions(model, solution_printer)

    return solution_printer.solutions

board_size = 4
solutions = solve_n_queens(board_size)

if solutions:
    print(f"Total solutions found: {len(solutions)}")
    for idx, solution in enumerate(solutions):
        print(f"Solution {idx+1}:")
        for row in range(board_size):
            line = ['.'] * board_size
            line[solution[row]] = 'Q'
            print(' '.join(line))
        print()
else:
    print("No solution exists.")


Total solutions found: 2
Solution 1:
. . Q .
Q . . .
. . . Q
. Q . .

Solution 2:
. Q . .
. . . Q
Q . . .
. . Q .

